In [1]:
from scipy.spatial.distance import pdist, squareform
from ripser import ripser
from ringity.new_centralities import current_distance as new_current_distance
from itertools import product

import ringity
import numpy as np
import os
import time
import networkx as nx

In [2]:
RINGITY_PATH = os.path.dirname(ringity.__file__)

In [3]:
G = nx.read_edgelist(f"{RINGITY_PATH}/test_data/lipid_coexp_network.csv",
                            nodetype=int)

In [4]:
ringity.get_distance_matrix(G, toa='induce', verbose=True)

'floyd_warshall' will be used for SPL calculation.
Current-distance will be induced.
Self loops in graph detected. They will be removed!
Time for current_flow_matrix calculation nr.0: 0.10440993309020996sec
Time for current_distance loop           nr.0: 2.98772931098938sec

Using edge attribute 'current-distance' as toa.
Time for Floyd-Warshall calculation: 0.06075286865234375sec


matrix([[0., 2., 2., ..., 4., 4., 4.],
        [2., 0., 2., ..., 4., 4., 4.],
        [2., 2., 0., ..., 3., 3., 3.],
        ...,
        [4., 4., 3., ..., 0., 1., 6.],
        [4., 4., 3., ..., 1., 0., 6.],
        [4., 4., 3., ..., 6., 6., 0.]])

In [5]:
nx.floyd_warshall_numpy(G, weight='current-distance')

matrix([[0., 2., 2., ..., 4., 4., 4.],
        [2., 0., 2., ..., 4., 4., 4.],
        [2., 2., 0., ..., 3., 3., 3.],
        ...,
        [4., 4., 3., ..., 0., 1., 6.],
        [4., 4., 3., ..., 1., 0., 6.],
        [4., 4., 3., ..., 6., 6., 0.]])

In [15]:
ringity.centralities.current_distance(G)

AttributeError: 'Graph' object has no attribute 'selfloop_edges'

In [4]:
ringity.diagram(G, verbose=True)

'floyd_warshall' will be used for SPL calculation.
Current-distance will be induced.
Self loops in graph detected. They will be removed!
Time for current_flow_matrix calculation nr.0: 0.10583901405334473sec
Time for current_distance loop           nr.0: 2.6247589588165283sec

Using edge attribute 'current-distance' as toa.
Time for Floyd-Warshall calculation: 0.06238913536071777sec
Time for ripser calculation: 0.21931910514831543sec


Dgm([(1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0), (1.0, 2.0)])

In [5]:
G[0][1]

{'current-distance': 15.097529008033764}

In [6]:
if induce:
    toa = 'induce'


D = get_distance_matrix(G          = G,
                        toa        = toa,
                        verbose    = verbose,
                        spl_method = spl_method,
                        split      = split)

In [13]:
D

matrix([[0., 2., 2., ..., 4., 4., 4.],
        [2., 0., 2., ..., 4., 4., 4.],
        [2., 2., 0., ..., 3., 3., 3.],
        ...,
        [4., 4., 3., ..., 0., 1., 6.],
        [4., 4., 3., ..., 1., 0., 6.],
        [4., 4., 3., ..., 6., 6., 0.]])

In [14]:
A = np.random.uniform(size=[244,3])
D2 = squareform(pdist(A))

In [35]:
D2 = D[:3,:3]
ripser(np.array(D2), distance_matrix=True)

{'dgms': [array([[ 0.,  2.],
         [ 0.,  2.],
         [ 0., inf]]), array([], shape=(0, 2), dtype=float64)],
 'cocycles': [[], []],
 'num_edges': 3,
 'dperm2all': array([[0., 2., 2.],
        [2., 0., 2.],
        [2., 2., 0.]]),
 'idx_perm': array([0, 1, 2]),
 'r_cover': 0.0}

In [34]:
np.array(D2)

array([[0., 2., 2.],
       [2., 0., 2.],
       [2., 2., 0.]])

In [11]:
t1 = time.time()
ripser_output = ripser(D, maxdim=p, distance_matrix=True)
dgm = Dgm(ripser_output['dgms'][p])
t2 = time.time()

if verbose:
    print(f'Time for ripser calculation: {t2-t1}sec')
return dgm

<class 'numpy.matrix'>



ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [23]:
for (i,j,k) in product(range(244),repeat=3):
    if D[i,j] + D[j,k] < D[i,k]:
        print(5)
        break

In [8]:
? nx.convert_node_labels_to_integers

In [6]:
G.nodes()

NodeView((0, 1, 2, 5, 10, 13, 14, 15, 17, 18, 19, 21, 27, 35, 41, 42, 57, 58, 59, 60, 62, 63, 64, 67, 70, 71, 79, 84, 85, 87, 92, 95, 100, 101, 102, 103, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 127, 136, 149, 154, 155, 156, 158, 166, 175, 178, 188, 192, 199, 213, 216, 222, 223, 224, 234, 148, 159, 11, 26, 37, 38, 66, 86, 91, 128, 132, 133, 137, 180, 183, 204, 205, 218, 6, 50, 51, 96, 126, 135, 139, 141, 142, 143, 20, 34, 129, 162, 118, 181, 237, 25, 43, 160, 196, 201, 16, 23, 24, 33, 36, 45, 48, 49, 74, 75, 76, 78, 80, 82, 83, 88, 89, 90, 93, 97, 98, 104, 140, 144, 145, 150, 151, 152, 157, 194, 229, 125, 29, 30, 31, 44, 52, 69, 94, 119, 198, 182, 12, 122, 176, 56, 228, 123, 153, 193, 65, 134, 207, 3, 4, 77, 68, 81, 105, 116, 138, 168, 53, 54, 61, 164, 212, 225, 243, 202, 203, 46, 55, 167, 7, 40, 28, 217, 147, 172, 173, 22, 117, 161, 163, 165, 195, 121, 210, 240, 32, 73, 99, 120, 124, 130, 131, 179, 186, 232, 47, 72, 174, 241, 169, 191, 219, 171, 206, 189, 208, 214, 238, 221, 